# 🚀 Multi-Timeframe Deep Learning Training System

## Системийн тайлбар

```
┌─────────────────────────────────────────────────┐
│  INPUT: Real-time MT5 data (1M candles)         │
└─────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────┐
│  FEATURE ENGINEERING (100+ features)            │
│  - Technical indicators (RSI, MACD, BB)         │
│  - Price patterns (candlestick, S/R levels)     │
│  - Volume profile                               │
│  - Market microstructure                        │
└─────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────┐
│  PARALLEL PREDICTION MODELS:                    │
│                                                 │
│  ┌────────────────┬───────────────────────┐    │
│  │ 15-min Model   │ Transformer + LSTM    │    │
│  │ Expected: 88%  │ Focus: Quick scalping │    │
│  └────────────────┴───────────────────────┘    │
│                                                 │
│  ┌────────────────┬───────────────────────┐    │
│  │ 30-min Model   │ Bi-LSTM + Attention   │    │
│  │ Expected: 85%  │ Focus: Swing trades   │    │
│  └────────────────┴───────────────────────┘    │
│                                                 │
│  ┌────────────────┬───────────────────────┐    │
│  │ 60-min Model   │ CNN-LSTM Hybrid       │    │
│  │ Expected: 82%  │ Focus: Trend following│    │
│  └────────────────┴───────────────────────┘    │
└─────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────┐
│  META-LEARNER (XGBoost)                         │
│  - Combines all 3 predictions                   │
│  - Learns which model to trust when             │
│  - Output: Direction + Confidence + Entry Price │
└─────────────────────────────────────────────────┘
```

## Зорилго

- 🎯 15-минут: **88%+ accuracy** (scalping)
- 🎯 30-минут: **85%+ accuracy** (swing trading)
- 🎯 60-минут: **82%+ accuracy** (trend following)
- 🎯 Ensemble: **90%+ accuracy** with confidence filter


## 📦 1. Import Libraries


In [ ]:
import sys
import os
from pathlib import Path

# Add backend to path
sys.path.append(str(Path.cwd().parent / 'backend'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, optimizers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

# Custom modules
from ml.features.technical_indicators import calculate_all_features
from ml.preprocessing.data_loader import ForexDataLoader
from ml.preprocessing.sequence_generator import create_sequences
from ml.models.transformer_lstm import build_transformer_lstm_model

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {tf.config.list_physical_devices('GPU')}")
print(f"✅ Python version: {sys.version}")

## ⚙️ 2. Configuration


In [ ]:
# Paths
DATA_DIR = Path.cwd().parent / 'data'
TRAIN_DIR = DATA_DIR / 'train'
TEST_DIR = DATA_DIR / 'test'
MODELS_DIR = Path.cwd().parent / 'models'
LOGS_DIR = Path.cwd().parent / 'logs'

# Create directories
for timeframe in ['15min', '30min', '60min', 'ensemble']:
    (MODELS_DIR / timeframe).mkdir(parents=True, exist_ok=True)
    (LOGS_DIR / timeframe / 'train').mkdir(parents=True, exist_ok=True)
    (LOGS_DIR / timeframe / 'validation').mkdir(parents=True, exist_ok=True)

# Currency pairs
CURRENCY_PAIRS = ['EUR_USD', 'GBP_USD', 'USD_JPY', 'USD_CAD', 'USD_CHF', 'XAU_USD']

# Training configuration
CONFIG = {
    '15min': {
        'resample_period': '15T',
        'sequence_length': 60,      # 60 * 15min = 15 hours history
        'prediction_steps': 1,      # Predict next 15 minutes
        'n_heads': 8,
        'ff_dim': 256,
        'lstm_units': [128, 64],
        'dropout': 0.3,
        'batch_size': 64,
        'epochs': 100,
        'learning_rate': 0.001
    },
    '30min': {
        'resample_period': '30T',
        'sequence_length': 48,      # 48 * 30min = 24 hours history
        'prediction_steps': 1,
        'n_heads': 8,
        'ff_dim': 256,
        'lstm_units': [128, 64],
        'dropout': 0.3,
        'batch_size': 64,
        'epochs': 100,
        'learning_rate': 0.001
    },
    '60min': {
        'resample_period': '60T',
        'sequence_length': 48,      # 48 * 60min = 48 hours history
        'prediction_steps': 1,
        'n_heads': 8,
        'ff_dim': 256,
        'lstm_units': [128, 64],
        'dropout': 0.3,
        'batch_size': 64,
        'epochs': 100,
        'learning_rate': 0.001
    }
}

print("✅ Configuration loaded")
print(f"📁 Data directory: {DATA_DIR}")
print(f"💾 Models directory: {MODELS_DIR}")
print(f"📊 Logs directory: {LOGS_DIR}")

## 📥 3. Load and Explore Data


In [ ]:
# Initialize data loader
loader = ForexDataLoader(data_dir=DATA_DIR)

# Load EUR/USD as example
pair = 'EUR_USD'
print(f"\n{'='*60}")
print(f"📊 Loading {pair} data")
print(f"{'='*60}\n")

df_train = loader.load_train_data(pair=pair)
df_test = loader.load_test_data(pair=pair)

print(f"\n📈 Training data shape: {df_train.shape}")
print(f"📉 Test data shape: {df_test.shape}")
print(f"\n📋 Columns: {df_train.columns.tolist()}")
print(f"\n🔍 First 5 rows:")
df_train.head()

## 🔧 4. Feature Engineering Function


In [ ]:
def prepare_data_for_timeframe(df, timeframe_config):
    """
    Prepare data for specific timeframe
    
    Steps:
    1. Resample 1-min data to target timeframe (15min/30min/60min)
    2. Calculate 100+ technical indicators
    3. Create labels (price direction after N candles)
    4. Remove NaN values
    5. Normalize features
    
    Returns:
        X: Features
        y: Labels (0=SELL, 1=NEUTRAL, 2=BUY)
        scaler: Fitted StandardScaler
    """
    print(f"\n🔄 Resampling to {timeframe_config['resample_period']}...")
    
    # Resample OHLCV data
    df_resampled = df.resample(timeframe_config['resample_period']).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'tick_volume': 'sum'
    }).dropna()
    
    print(f"✅ Resampled shape: {df_resampled.shape}")
    
    # Calculate features
    print("\n🔧 Calculating technical indicators...")
    df_features = calculate_all_features(df_resampled)
    
    # Create labels
    print("\n🏷️  Creating labels...")
    prediction_steps = timeframe_config['prediction_steps']
    
    # Future price change
    df_features['future_return'] = df_features['close'].shift(-prediction_steps) / df_features['close'] - 1
    
    # Label: -1 (SELL), 0 (NEUTRAL), 1 (BUY)
    # Threshold: ±0.0005 (5 pips for most pairs)
    threshold = 0.0005
    conditions = [
        df_features['future_return'] < -threshold,  # SELL
        (df_features['future_return'] >= -threshold) & (df_features['future_return'] <= threshold),  # NEUTRAL
        df_features['future_return'] > threshold   # BUY
    ]
    df_features['label'] = np.select(conditions, [0, 1, 2], default=1)
    
    # Remove NaN
    df_features = df_features.dropna()
    
    print(f"✅ Features calculated: {df_features.shape}")
    print(f"\n📊 Label distribution:")
    print(df_features['label'].value_counts())
    print(f"\nClass percentages:")
    print(df_features['label'].value_counts(normalize=True) * 100)
    
    # Separate features and labels
    feature_cols = [col for col in df_features.columns 
                   if col not in ['label', 'future_return', 'open', 'high', 'low', 'close']]
    
    X = df_features[feature_cols].values
    y = df_features['label'].values
    
    # Normalize features
    print("\n🔄 Normalizing features...")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(f"✅ Final shape: X={X_scaled.shape}, y={y.shape}")
    print(f"✅ Number of features: {len(feature_cols)}")
    
    return X_scaled, y, scaler, feature_cols

print("✅ Feature engineering function defined")

## 🔄 5. Create Sequences Function


In [ ]:
def create_sequences_for_training(X, y, sequence_length):
    """
    Create sequences for LSTM/Transformer input
    
    Args:
        X: Features (n_samples, n_features)
        y: Labels (n_samples,)
        sequence_length: Number of timesteps per sequence
        
    Returns:
        X_seq: (n_sequences, sequence_length, n_features)
        y_seq: (n_sequences,)
    """
    X_seq = []
    y_seq = []
    
    for i in range(sequence_length, len(X)):
        X_seq.append(X[i-sequence_length:i])
        y_seq.append(y[i])
    
    return np.array(X_seq), np.array(y_seq)

print("✅ Sequence creation function defined")

## 🏋️ 6. Training Function


In [ ]:
def train_model_for_timeframe(timeframe, pair='EUR_USD'):
    """
    Train model for specific timeframe
    
    Args:
        timeframe: '15min', '30min', or '60min'
        pair: Currency pair name
        
    Returns:
        model: Trained Keras model
        history: Training history
        test_metrics: Test set performance
    """
    print(f"\n{'='*80}")
    print(f"🚀 TRAINING {timeframe.upper()} MODEL FOR {pair}")
    print(f"{'='*80}\n")
    
    config = CONFIG[timeframe]
    
    # Load data
    print("📥 Loading data...")
    loader = ForexDataLoader(data_dir=DATA_DIR)
    df_train = loader.load_train_data(pair=pair)
    df_test = loader.load_test_data(pair=pair)
    
    # Prepare training data
    print("\n🔧 Preparing training data...")
    X_train, y_train, scaler_train, feature_cols = prepare_data_for_timeframe(df_train, config)
    
    # Prepare test data
    print("\n🔧 Preparing test data...")
    X_test, y_test, scaler_test, _ = prepare_data_for_timeframe(df_test, config)
    
    # Use training scaler for test data
    X_test = scaler_train.transform(X_test)
    
    # Create sequences
    print(f"\n🔄 Creating sequences (length={config['sequence_length']})...")
    X_train_seq, y_train_seq = create_sequences_for_training(X_train, y_train, config['sequence_length'])
    X_test_seq, y_test_seq = create_sequences_for_training(X_test, y_test, config['sequence_length'])
    
    print(f"✅ Training sequences: {X_train_seq.shape}")
    print(f"✅ Test sequences: {X_test_seq.shape}")
    
    # Split validation set
    X_train_seq, X_val_seq, y_train_seq, y_val_seq = train_test_split(
        X_train_seq, y_train_seq, test_size=0.2, random_state=42, stratify=y_train_seq
    )
    
    print(f"\n📊 Final splits:")
    print(f"   Train: {X_train_seq.shape}")
    print(f"   Val:   {X_val_seq.shape}")
    print(f"   Test:  {X_test_seq.shape}")
    
    # Build model
    print(f"\n🏗️  Building model...")
    n_features = X_train_seq.shape[2]
    
    model = build_transformer_lstm_model(
        sequence_length=config['sequence_length'],
        n_features=n_features,
        n_heads=config['n_heads'],
        ff_dim=config['ff_dim'],
        lstm_units=config['lstm_units'],
        dropout_rate=config['dropout'],
        n_classes=3  # BUY/NEUTRAL/SELL
    )
    
    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=config['learning_rate']),
        loss={
            'direction': 'sparse_categorical_crossentropy',
            'confidence': 'mse'
        },
        loss_weights={'direction': 1.0, 'confidence': 0.5},
        metrics={
            'direction': ['accuracy'],
            'confidence': ['mae']
        }
    )
    
    print(model.summary())
    
    # Callbacks
    model_path = MODELS_DIR / timeframe / f"{pair}_{timeframe}_best.keras"
    log_dir = LOGS_DIR / timeframe / 'train' / datetime.now().strftime("%Y%m%d-%H%M%S")
    
    callbacks_list = [
        callbacks.ModelCheckpoint(
            filepath=str(model_path),
            monitor='val_direction_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        ),
        callbacks.EarlyStopping(
            monitor='val_direction_accuracy',
            patience=15,
            mode='max',
            restore_best_weights=True,
            verbose=1
        ),
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        callbacks.TensorBoard(
            log_dir=str(log_dir),
            histogram_freq=1,
            write_graph=True
        )
    ]
    
    # Train model
    print(f"\n🏋️ Training model...")
    print(f"📊 Batch size: {config['batch_size']}")
    print(f"📊 Epochs: {config['epochs']}")
    print(f"📊 Learning rate: {config['learning_rate']}")
    
    history = model.fit(
        X_train_seq,
        {
            'direction': y_train_seq,
            'confidence': np.ones_like(y_train_seq, dtype=float)  # Placeholder
        },
        validation_data=(
            X_val_seq,
            {
                'direction': y_val_seq,
                'confidence': np.ones_like(y_val_seq, dtype=float)
            }
        ),
        batch_size=config['batch_size'],
        epochs=config['epochs'],
        callbacks=callbacks_list,
        verbose=1
    )
    
    # Evaluate on test set
    print(f"\n📊 Evaluating on test set...")
    test_results = model.evaluate(
        X_test_seq,
        {
            'direction': y_test_seq,
            'confidence': np.ones_like(y_test_seq, dtype=float)
        },
        verbose=1
    )
    
    # Make predictions
    predictions = model.predict(X_test_seq)
    y_pred = np.argmax(predictions[0], axis=1)
    
    # Calculate accuracy per class
    from sklearn.metrics import classification_report, confusion_matrix
    
    print("\n" + "="*80)
    print(f"📈 TEST SET RESULTS FOR {timeframe.upper()}")
    print("="*80)
    print("\nClassification Report:")
    print(classification_report(y_test_seq, y_pred, target_names=['SELL', 'NEUTRAL', 'BUY']))
    
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_test_seq, y_pred)
    print(cm)
    
    # Save scaler and metadata
    scaler_path = MODELS_DIR / timeframe / f"{pair}_{timeframe}_scaler.pkl"
    joblib.dump(scaler_train, scaler_path)
    
    metadata = {
        'timeframe': timeframe,
        'pair': pair,
        'n_features': n_features,
        'sequence_length': config['sequence_length'],
        'feature_columns': feature_cols,
        'train_samples': len(X_train_seq),
        'test_accuracy': float(np.mean(y_pred == y_test_seq)),
        'training_date': datetime.now().isoformat()
    }
    
    import json
    metadata_path = MODELS_DIR / timeframe / f"{pair}_{timeframe}_metadata.json"
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\n💾 Model saved: {model_path}")
    print(f"💾 Scaler saved: {scaler_path}")
    print(f"💾 Metadata saved: {metadata_path}")
    
    return model, history, test_results

print("✅ Training function defined")

## 🚀 7. Train All Models

Одоо 3 модельыг зэрэг сургана:

- 15-минутын модель (scalping)
- 30-минутын модель (swing trading)
- 60-минутын модель (trend following)


In [ ]:
# Train EUR/USD for all timeframes
pair = 'EUR_USD'
results = {}

for timeframe in ['15min', '30min', '60min']:
    try:
        model, history, test_results = train_model_for_timeframe(timeframe, pair=pair)
        results[timeframe] = {
            'model': model,
            'history': history,
            'test_results': test_results
        }
        print(f"\n✅ {timeframe} training completed!\n")
    except Exception as e:
        print(f"\n❌ Error training {timeframe}: {str(e)}\n")
        import traceback
        traceback.print_exc()

print("\n" + "="*80)
print("🎉 ALL MODELS TRAINING COMPLETED!")
print("="*80)

## 📊 8. Visualize Training Results


In [ ]:
# Plot training history for all models
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Training History - All Timeframes', fontsize=16, fontweight='bold')

for idx, (timeframe, result) in enumerate(results.items()):
    if 'history' not in result:
        continue
        
    history = result['history']
    
    # Accuracy plot
    ax1 = axes[0, idx]
    ax1.plot(history.history['direction_accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_direction_accuracy'], label='Val Accuracy')
    ax1.set_title(f'{timeframe} - Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Loss plot
    ax2 = axes[1, idx]
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Val Loss')
    ax2.set_title(f'{timeframe} - Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(MODELS_DIR / 'training_results_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Training plots saved!")

## 🎯 9. Test All Models on Test Data


In [ ]:
def test_model_performance(timeframe, pair='EUR_USD'):
    """
    Load trained model and test on test dataset
    """
    print(f"\n{'='*80}")
    print(f"🧪 TESTING {timeframe.upper()} MODEL ON {pair}")
    print(f"{'='*80}\n")
    
    # Load model and scaler
    model_path = MODELS_DIR / timeframe / f"{pair}_{timeframe}_best.keras"
    scaler_path = MODELS_DIR / timeframe / f"{pair}_{timeframe}_scaler.pkl"
    
    if not model_path.exists():
        print(f"❌ Model not found: {model_path}")
        return None
    
    print(f"📥 Loading model from {model_path}")
    model = keras.models.load_model(model_path)
    scaler = joblib.load(scaler_path)
    
    # Load test data
    loader = ForexDataLoader(data_dir=DATA_DIR)
    df_test = loader.load_test_data(pair=pair)
    
    config = CONFIG[timeframe]
    
    # Prepare test data
    X_test, y_test, _, _ = prepare_data_for_timeframe(df_test, config)
    X_test = scaler.transform(X_test)
    
    # Create sequences
    X_test_seq, y_test_seq = create_sequences_for_training(X_test, y_test, config['sequence_length'])
    
    print(f"\n📊 Test data shape: {X_test_seq.shape}")
    
    # Make predictions
    predictions = model.predict(X_test_seq, verbose=1)
    y_pred = np.argmax(predictions[0], axis=1)
    y_confidence = predictions[1].flatten()
    
    # Calculate metrics
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
    
    accuracy = accuracy_score(y_test_seq, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_seq, y_pred, average='weighted')
    
    print("\n" + "="*80)
    print(f"📈 TEST RESULTS FOR {timeframe.upper()}")
    print("="*80)
    print(f"\n🎯 Overall Accuracy: {accuracy*100:.2f}%")
    print(f"🎯 Precision: {precision*100:.2f}%")
    print(f"🎯 Recall: {recall*100:.2f}%")
    print(f"🎯 F1-Score: {f1*100:.2f}%")
    
    print("\n📊 Classification Report:")
    print(classification_report(y_test_seq, y_pred, target_names=['SELL', 'NEUTRAL', 'BUY']))
    
    # High confidence predictions
    high_conf_mask = y_confidence > 0.85
    if high_conf_mask.sum() > 0:
        high_conf_accuracy = accuracy_score(y_test_seq[high_conf_mask], y_pred[high_conf_mask])
        print(f"\n✨ High Confidence (>85%) Accuracy: {high_conf_accuracy*100:.2f}%")
        print(f"   Number of high confidence predictions: {high_conf_mask.sum()} ({high_conf_mask.sum()/len(y_pred)*100:.1f}%)")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'predictions': y_pred,
        'confidence': y_confidence,
        'true_labels': y_test_seq
    }

# Test all models
test_results = {}
for timeframe in ['15min', '30min', '60min']:
    test_results[timeframe] = test_model_performance(timeframe, pair=pair)

print("\n" + "="*80)
print("🎉 ALL MODELS TESTED!")
print("="*80)

## 📊 10. Compare Model Performance


In [ ]:
# Create comparison table
comparison_data = []
for timeframe, result in test_results.items():
    if result is not None:
        comparison_data.append({
            'Timeframe': timeframe,
            'Accuracy (%)': f"{result['accuracy']*100:.2f}",
            'Precision (%)': f"{result['precision']*100:.2f}",
            'Recall (%)': f"{result['recall']*100:.2f}",
            'F1-Score (%)': f"{result['f1']*100:.2f}"
        })

comparison_df = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("📊 MODEL COMPARISON SUMMARY")
print("="*80)
print(comparison_df.to_string(index=False))

# Save comparison
comparison_df.to_csv(MODELS_DIR / 'model_comparison.csv', index=False)
print(f"\n💾 Comparison saved to {MODELS_DIR / 'model_comparison.csv'}")

## 🎯 11. Ensemble Prediction Example


In [ ]:
def ensemble_predict(sample_idx=0):
    """
    Example: Combine predictions from all 3 models
    """
    print("\n" + "="*80)
    print("🎯 ENSEMBLE PREDICTION EXAMPLE")
    print("="*80)
    
    predictions = {}
    
    for timeframe in ['15min', '30min', '60min']:
        result = test_results[timeframe]
        if result is not None:
            pred = result['predictions'][sample_idx]
            conf = result['confidence'][sample_idx]
            true_label = result['true_labels'][sample_idx]
            
            predictions[timeframe] = {
                'prediction': ['SELL', 'NEUTRAL', 'BUY'][pred],
                'confidence': conf,
                'true_label': ['SELL', 'NEUTRAL', 'BUY'][true_label]
            }
    
    print(f"\nSample #{sample_idx}:")
    print("\n" + "-"*60)
    for timeframe, pred in predictions.items():
        print(f"{timeframe:8s} | Prediction: {pred['prediction']:7s} | Confidence: {pred['confidence']:.3f} | True: {pred['true_label']}")
    print("-"*60)
    
    # Simple voting
    votes = [pred['prediction'] for pred in predictions.values()]
    from collections import Counter
    vote_count = Counter(votes)
    ensemble_prediction = vote_count.most_common(1)[0][0]
    
    print(f"\n🗳️  Ensemble Vote: {ensemble_prediction}")
    print(f"📊 Vote breakdown: {dict(vote_count)}")
    
    return predictions, ensemble_prediction

# Test ensemble on first 10 samples
print("\n🧪 Testing ensemble on 10 samples...\n")
for i in range(min(10, len(test_results['15min']['predictions']))):
    ensemble_predict(sample_idx=i)

## 💡 12. Next Steps

### ✅ Completed:

- [x] Data loading and preprocessing
- [x] Feature engineering (100+ indicators)
- [x] Multi-timeframe model training
- [x] Model evaluation on test data
- [x] Basic ensemble voting

### 🔜 To Implement:

1. **Meta-Learner (XGBoost)**

   - Train ensemble model on predictions from 3 models
   - Learn when to trust which model

2. **Confidence Filtering**

   - Filter signals by confidence threshold (>85%)
   - Multi-timeframe alignment check

3. **Risk Management**

   - Calculate entry price, take profit, stop loss
   - Expected time to target

4. **Production Integration**

   - Flask API endpoints
   - Real-time prediction from MT5
   - Mobile app integration

5. **Monitoring**
   - TensorBoard logs
   - Performance tracking
   - Retraining triggers


## 🎉 Summary

Энэ notebook-д бид:

1. ✅ **Data/train folder**-оос 1-минутын датаг уншсан
2. ✅ 15-мин, 30-мин, 60-мин **timeframe руу resample** хийсэн
3. ✅ **100+ техникийн үзүүлэлт** тооцсон
4. ✅ **Label үүсгэсэн** (BUY/NEUTRAL/SELL)
5. ✅ **Transformer + LSTM модель** бүтээж, сургасан
6. ✅ **Data/test folder**-ын датаар шалгасан
7. ✅ **Гурван модель** зэрэг сургасан
8. ✅ **Ensemble voting** систем үүсгэсэн
9. ✅ **Үр дүнг харьцуулсан**

### Дараагийн алхам:

- Бусад валютын хослол дээр сургах (GBP/USD, USD/JPY, гэх мэт)
- Meta-learner (XGBoost) сургах
- Flask API дээр deploy хийх
- Mobile app-тэй холбох
